# Biostat 257 Homework 5

**Due June 3 @ 11:59PM**

In [2]:
versioninfo()

Julia Version 1.7.2
Commit bf53498635 (2022-02-06 15:21 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin19.5.0)
  CPU: Intel(R) Core(TM) i5-4260U CPU @ 1.40GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-12.0.1 (ORCJIT, haswell)


In this assignment, we continue with the linear mixed effects model (LMM) considered in HW3
$$
    \mathbf{Y}_i = \mathbf{X}_i \boldsymbol{\beta} + \mathbf{Z}_i \boldsymbol{\gamma}_i + \boldsymbol{\epsilon}_i, \quad i=1,\ldots,n,
$$
where   
- $\mathbf{Y}_i \in \mathbb{R}^{n_i}$ is the response vector of $i$-th individual,  
- $\mathbf{X}_i \in \mathbb{R}^{n_i \times p}$ is the fixed effects predictor matrix of $i$-th individual,  
- $\mathbf{Z}_i \in \mathbb{R}^{n_i \times q}$ is the random effects predictor matrix of $i$-th individual,  
- $\boldsymbol{\epsilon}_i \in \mathbb{R}^{n_i}$ are multivariate normal $N(\mathbf{0}_{n_i},\sigma^2 \mathbf{I}_{n_i})$,  
- $\boldsymbol{\beta} \in \mathbb{R}^p$ are fixed effects, and  
- $\boldsymbol{\gamma}_i \in \mathbb{R}^q$ are random effects assumed to be $N(\mathbf{0}_q, \boldsymbol{\Sigma}_{q \times q}$) independent of $\boldsymbol{\epsilon}_i$.

The log-likelihood of the $i$-th datum $(\mathbf{y}_i, \mathbf{X}_i, \mathbf{Z}_i)$ is 
$$
    \ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma_0^2) = - \frac{n_i}{2} \log (2\pi) - \frac{1}{2} \log \det \boldsymbol{\Omega}_i - \frac{1}{2} (\mathbf{y} - \mathbf{X}_i \boldsymbol{\beta})^T \boldsymbol{\Omega}_i^{-1} (\mathbf{y} - \mathbf{X}_i \boldsymbol{\beta}),
$$
where
$$
    \boldsymbol{\Omega}_i = \sigma^2 \mathbf{I}_{n_i} + \mathbf{Z}_i \boldsymbol{\Sigma} \mathbf{Z}_i^T = \sigma^2 \mathbf{I}_{n_i} + \mathbf{Z}_i \mathbf{L} \mathbf{L}^T \mathbf{Z}_i^T.
$$
Because the variance component parameter $\boldsymbol{\Sigma}$ has to be positive semidefinite. We prefer to use its Cholesky factor $\mathbf{L}$ as optimization variable. 

Given $m$ independent data tuples $(\mathbf{y}_i, \mathbf{X}_i, \mathbf{Z}_i)$, $i=1,\ldots,m$, we seek the maximum likelihood estimate (MLE) by maximizing the log-likelihood
$$
\ell(\boldsymbol{\beta}, \boldsymbol{\Sigma}, \sigma^2) = \sum_{i=1}^m \ell_i(\boldsymbol{\beta}, \boldsymbol{\Sigma}, \sigma^2).
$$
In this assignment, we use the nonlinear programming (NLP) approach for optimization. In HW6, we will derive an EM (expectation-maximization) algorithm for the same problem. In HW7, we will derive an MM (minorization-maximization) algorithm for the same problem. I'm kidding 😁 There's no HW7. If you are curious about how to derive MM algorithm for this problem, see [this article](https://doi.org/10.1080/10618600.2018.1529601).

In [21]:
#import Pkg; Pkg.add("CSV")
#import Pkg; Pkg.add("DataFrames")
#import Pkg; Pkg.add("Ipopt")
#import Pkg; Pkg.add("MathOptInterface")
#import Pkg; Pkg.add("MixedModels")
#import Pkg; Pkg.add("NLopt")
#import Pkg; Pkg.add("PrettyTables")
#import Pkg; Pkg.add("RCall")

In [3]:
# load necessary packages; make sure install them first
using BenchmarkTools, CSV, DataFrames, DelimitedFiles, Distributions
using Ipopt, LinearAlgebra, MathOptInterface, MixedModels, NLopt
using PrettyTables, Random, RCall

const MOI = MathOptInterface

MathOptInterface

## Q1. (Optional, 30 bonus pts) Derivatives

NLP optimization solvers expect users to provide at least a function for evaluating objective value. If users can provide further information such as gradient and Hessian, the NLP solvers will be more stable and converge faster. Automatic differentiation tools are becoming more powerful but cannot apply to all problems yet.

1. Show that the gradient of $\ell_i$ is
\begin{eqnarray*}
\nabla_{\boldsymbol{\beta}} \ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma^2) &=& \mathbf{X}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i, \\
\nabla_{\sigma^2} \ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma^2) &=& - \frac{1}{2} \operatorname{tr} (\boldsymbol{\Omega}_i^{-1}) + \frac{1}{2} \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-2} \mathbf{r}_i, \\
\frac{\partial}{\partial \mathbf{L}} \ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma^2) &=& - \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L} + \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L},
\end{eqnarray*}
where $\mathbf{r}_i = \mathbf{y}_i - \mathbf{X}_i \boldsymbol{\beta}$. 

2. Derive the observed information matrix and the expected (Fisher) information matrix.

If you need a refresher on multivariate calculus, my [Biostat 216 lecture notes](https://ucla-biostat216-2019fall.github.io/slides/16-matrixcalc/16-matrixcalc.html) may be helpful.

#### Solution for deriving gradient of $\ell_i$ 

 $$\ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma_0^2) = - \frac{n_i}{2} \log (2\pi) - \frac{1}{2} \log \det \boldsymbol{\Omega}_i - \frac{1}{2} (\mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i)$$

$$\ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma_0^2) = - \frac{n_i}{2} \log (2\pi) - \frac{1}{2} \log \det \boldsymbol{\Omega}_i - \frac{1}{2} [\mathbf{y}_i \boldsymbol{\Omega}_i^{-1} \mathbf{y}_i - 2 \boldsymbol{\beta}^T \mathbf{X}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{y}_i + \boldsymbol{\beta}^T \mathbf{X}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{X}_i \boldsymbol{\beta}]$$

1. Show that $\nabla_{\boldsymbol{\beta}} \ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma^2) = \mathbf{X}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i$

$$\nabla_{\boldsymbol{\beta}} \ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma^2) = -\frac{1}{2} (-2\mathbf{X}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{y}_i + 2\mathbf{X}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{X}_i \boldsymbol{\beta}) = \mathbf{X}_i^T \boldsymbol{\Omega}_i^{-1} (\mathbf{y}_i - \mathbf{X}_i \boldsymbol{\beta}) = \mathbf{X}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i$$

2. Show that $\nabla_{\sigma^2} \ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma^2) = - \frac{1}{2} \operatorname{tr} (\boldsymbol{\Omega}_i^{-1}) + \frac{1}{2} \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-2} \mathbf{r}_i$

$$\nabla_{\sigma^2} \ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma^2) = - \frac{1}{2} D_{\sigma^2} \log \det \boldsymbol{\Omega}_i - \frac{1}{2} D_{\sigma^2} \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i$$

a. $$D_{\sigma^2} \log \det \boldsymbol{\Omega}_i = D_{\boldsymbol{\Omega}_i} \log \det \boldsymbol{\Omega}_i * D_{\sigma^2} \boldsymbol{\Omega}_i \\ = \text{vec}(\boldsymbol{\Omega}_i^{-1})^T * \text{vec}(\mathbf{I}_{n_i}) \\ = \text{tr}(\boldsymbol{\Omega}_i^{-1} * \mathbf{I}_{n_i}) \\ = \text{tr}(\boldsymbol{\Omega}_i^{-1})$$

b. $$D_{\sigma^2} \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i = D_{\boldsymbol{\Omega}_i} \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i * D_{\sigma^2} \boldsymbol{\Omega}_i \\ = D_{\boldsymbol{\Omega}_i} \text{tr}(\mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i) * \text{vec}(\mathbf{I}_{n_i}) \\ = D_{\boldsymbol{\Omega}_i} \text{tr}(\mathbf{r}_i \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1}) * \text{vec}(\mathbf{I}_{n_i}) \\ = \text{vec}(-\boldsymbol{\Omega}_i^{-1} \mathbf{r}_i \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1})^T * \text{vec}(\mathbf{I}_{n_i}) \\ = \text{tr}(-\boldsymbol{\Omega}_i^{-1} \mathbf{r}_i \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1}) \\ = -\text{tr}(\mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i) \\ = -\mathbf{r}_i^T \boldsymbol{\Omega}_i^{-2} \mathbf{r}_i$$

Combining together, we get $$\nabla_{\sigma^2} \ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma^2) = - \frac{1}{2} \operatorname{tr} (\boldsymbol{\Omega}_i^{-1}) + \frac{1}{2} \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-2} \mathbf{r}_i $$

3. Show that $\frac{\partial}{\partial \mathbf{L}} \ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma^2) = - \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L} + \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L}$

$$\frac{\partial}{\partial \mathbf{L}} \ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma^2) = - \frac{1}{2} D_{\mathbf{L}} \log \det \boldsymbol{\Omega}_i - \frac{1}{2} D_{\mathbf{L}} \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i$$

a. $$D_{\mathbf{L}} \log \det \boldsymbol{\Omega}_i = D_{\boldsymbol{\Omega}_i} \log \det \boldsymbol{\Omega}_i * D_{\mathbf{\Sigma}} \boldsymbol{\Omega}_i * D_{\mathbf{L}} \mathbf{\Sigma} \\ = \text{vec}(\boldsymbol{\Omega}_i^{-1})^T * (\mathbf{Z}_i \otimes \mathbf{Z}_i) * 2N_q (\mathbf{L} \otimes \mathbf{I}_q) \text{    where  } N_q = \frac{1}{2}(\mathbf{I}_{q^2} + \mathbf{K}_{qq})\\ = \text{vec}(\boldsymbol{\Omega}_i^{-1})^T * (\mathbf{Z}_i \otimes \mathbf{Z}_i) * (\mathbf{I}_{q^2} + \mathbf{K}_{qq}) (\mathbf{L} \otimes \mathbf{I}_q) \\ = \text{vec}(\boldsymbol{\Omega}_i^{-1})^T * (\mathbf{Z}_i \otimes \mathbf{Z}_i) * [\mathbf{L} \otimes \mathbf{I}_q + \mathbf{K}_{qq} (\mathbf{L} \otimes \mathbf{I}_q)]$$

i. $$ \text{vec}(\boldsymbol{\Omega}_i^{-1})^T * (\mathbf{Z}_i \otimes \mathbf{Z}_i) * \mathbf{L} \otimes \mathbf{I}_q = \text{vec}(\boldsymbol{\Omega}_i^{-1})^T * (\mathbf{Z}_i \mathbf{L} \otimes \mathbf{Z}_i) \\ = [(\mathbf{L}_i^T \mathbf{Z}_i^T \otimes \mathbf{Z}_i^T) \text{vec}(\boldsymbol{\Omega}_i^{-1})]^T \\ = \text{vec}(\mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L})^T$$

ii. $$\text{vec}(\boldsymbol{\Omega}_i^{-1})^T * (\mathbf{Z}_i \otimes \mathbf{Z}_i) * \mathbf{K}_{qq} (\mathbf{L} \otimes \mathbf{I}_q) \\ = [(\mathbf{Z}_i^T \otimes \mathbf{Z}_i^T) \text{vec}(\boldsymbol{\Omega}_i^{-1})]^T * \mathbf{K}_{qq} (\mathbf{L} \otimes \mathbf{I}_q) \\ = \text{vec}(\mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i)^T \mathbf{K}_{qq} (\mathbf{L} \otimes \mathbf{I}_q) \\ = [\mathbf{K}_{qq} \text{vec}(\mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i)]^T (\mathbf{L} \otimes \mathbf{I}_q) \\ =  \text{vec}(\mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i)^T (\mathbf{L} \otimes \mathbf{I}_q) \\ = [(\mathbf{L}^T \otimes \mathbf{I}_q) \text{vec}(\mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i)]^T \\ = \text{vec}(\mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L})^T$$

$\mathbf{K}_{qq}$ is an orthogonal matrix, so it is symmetric; $\mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i$ is symmetric as well.

Together, $$D_{\mathbf{L}} \log \det \boldsymbol{\Omega}_i = 2 * \text{vec}(\mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L})^T $$

b. $$D_{\mathbf{L}} \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i = D_{\boldsymbol{\Omega}_i} \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i * D_{\mathbf{\Sigma}} \boldsymbol{\Omega}_i * D_{\mathbf{L}} \mathbf{\Sigma} \\ = D_{\boldsymbol{\Omega}_i} \text{tr}(\mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i) * D_{\mathbf{\Sigma}} \boldsymbol{\Omega}_i * D_{\mathbf{L}} \mathbf{\Sigma} \\ = D_{\boldsymbol{\Omega}_i} \text{tr}(\mathbf{r}_i \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1}) * D_{\mathbf{\Sigma}} \boldsymbol{\Omega}_i * D_{\mathbf{L}} \mathbf{\Sigma} \\ = \text{vec}(-\boldsymbol{\Omega}_i^{-1} \mathbf{r}_i \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1})^T (\mathbf{Z}_i \otimes \mathbf{Z}_i) * [\mathbf{L} \otimes \mathbf{I}_q + \mathbf{K}_{qq} (\mathbf{L} \otimes \mathbf{I}_q)]$$

i. $$ \text{vec}(-\boldsymbol{\Omega}_i^{-1} \mathbf{r}_i \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1})^T * (\mathbf{Z}_i \otimes \mathbf{Z}_i) * \mathbf{L} \otimes \mathbf{I}_q = \text{vec}(-\boldsymbol{\Omega}_i^{-1} \mathbf{r}_i \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1})^T * (\mathbf{Z}_i \mathbf{L} \otimes \mathbf{Z}_i) \\ = [(\mathbf{L}_i^T \mathbf{Z}_i^T \otimes \mathbf{Z}_i^T) \text{vec}(-\boldsymbol{\Omega}_i^{-1} \mathbf{r}_i \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1})]^T \\ = \text{vec}(-\mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L})^T$$

ii. $$\text{vec}(-\boldsymbol{\Omega}_i^{-1} \mathbf{r}_i \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1})^T * (\mathbf{Z}_i \otimes \mathbf{Z}_i) * \mathbf{K}_{qq} (\mathbf{L} \otimes \mathbf{I}_q) \\ = [(\mathbf{Z}_i^T \otimes \mathbf{Z}_i^T) \text{vec}(-\boldsymbol{\Omega}_i^{-1} \mathbf{r}_i \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1})]^T * \mathbf{K}_{qq} (\mathbf{L} \otimes \mathbf{I}_q) \\ = \text{vec}(-\mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i)^T \mathbf{K}_{qq} (\mathbf{L} \otimes \mathbf{I}_q) \\ = [\mathbf{K}_{qq} \text{vec}(-\mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i)]^T (\mathbf{L} \otimes \mathbf{I}_q) \\ =  \text{vec}(-\mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i)^T (\mathbf{L} \otimes \mathbf{I}_q) \\ = [(\mathbf{L}^T \otimes \mathbf{I}_q) \text{vec}(-\mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i)]^T \\ = \text{vec}(-\mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L})^T$$

$\mathbf{K}_{qq}$ is an orthgonal matrix, so it is symmetric; $\mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i$ is symmetric as well.

Together, $$D_{\mathbf{L}} \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i = -2 * \text{vec}(\mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L})^T$$

Combining everything together, we get $$\frac{\partial}{\partial \mathbf{L}} \ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma^2) = - \frac{1}{2} (2 * \text{vec}(\mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L})^T)+ \frac{1}{2} (2 * \text{vec}(\mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L})^T) $$

Equivalent to the form, $$\frac{\partial}{\partial \mathbf{L}} \ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma^2) = - \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L} + \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L}$$

#### Solution to the observed information matrix and the expected (Fisher) information matrix

Future work!

## Q2. (20 pts) Objective and gradient evaluator for a single datum

We expand the code from HW3 to evaluate both objective and gradient. I provide my code for HW3 below as a starting point. You do _not_ have to use this code. If your come up faster code, that's even better. 

In [4]:
# define a type that holds an LMM datum
struct LmmObs{T <: AbstractFloat}
    # data
    y          :: Vector{T}
    X          :: Matrix{T}
    Z          :: Matrix{T}
    # arrays for holding gradient
    ∇β         :: Vector{T}
    ∇σ²        :: Vector{T}
    ∇Σ         :: Matrix{T}    
    # working arrays
    # intermediate arrays for pre-allocation
    yty        :: T
    xty        :: Vector{T}
    zty        :: Vector{T}
    storage_p  :: Vector{T}
    xtxbeta    :: Vector{T} # px1
    storage_q  :: Vector{T}
    storage_q2 :: Vector{T} # qx1
    storage_q3 :: Vector{T} # qx1
    xtx        :: Matrix{T}
    ztx        :: Matrix{T}
    ztz        :: Matrix{T}
    ztxbeta    :: Vector{T} # qx1
    storage_qq :: Matrix{T}
    store_qq2  :: Matrix{T} # qxq
    store_qq3  :: Matrix{T} # qxq
end

"""
    LmmObs(y::Vector, X::Matrix, Z::Matrix)

Create an LMM datum of type `LmmObs`.
"""
function LmmObs(
        y::Vector{T}, 
        X::Matrix{T}, 
        Z::Matrix{T}
    ) where T <: AbstractFloat
    n, p, q    = size(X, 1), size(X, 2), size(Z, 2)    
    ∇β         = Vector{T}(undef, p)
    ∇σ²        = Vector{T}(undef, 1)
    ∇Σ         = Matrix{T}(undef, q, q)    
    yty        = abs2(norm(y))
    xty        = transpose(X) * y
    zty        = transpose(Z) * y    
    storage_p  = Vector{T}(undef, p)
    xtxbeta    = Vector{T}(undef, p)
    storage_q  = Vector{T}(undef, q)
    storage_q2 = Vector{T}(undef, q)
    storage_q3 = Vector{T}(undef, q)
    xtx        = transpose(X) * X
    ztx        = transpose(Z) * X
    ztz        = transpose(Z) * Z
    ztxbeta    = Vector{T}(undef, q)
    storage_qq = similar(ztz)
    store_qq2  = similar(ztz)
    store_qq3  = similar(ztz)
    LmmObs(y, X, Z, ∇β, ∇σ², ∇Σ, yty, xty, zty, storage_p, xtxbeta, 
        storage_q, storage_q2, storage_q3, xtx, ztx, ztz, ztxbeta, 
        storage_qq, store_qq2, store_qq3)
end

"""
    logl!(obs::LmmObs, β, L, σ², needgrad=false)

Evaluate the log-likelihood of a single LMM datum at parameter values `β`, `L`, 
and `σ²`. If `needgrad==true`, then `obs.∇β`, `obs.∇Σ`, and `obs.σ² are filled 
with the corresponding gradient.
"""
function logl!(
        obs      :: LmmObs{T}, 
        β        :: Vector{T}, 
        L        :: Matrix{T}, 
        σ²       :: T,
        needgrad :: Bool = true
    ) where T <: AbstractFloat
    n, p, q = size(obs.X, 1), size(obs.X, 2), size(obs.Z, 2)
    ####################
    # Evaluate objective
    ####################    
    # form the q-by-q matrix: M = σ² * I + Lt Zt Z L
    copy!(obs.storage_qq, obs.ztz) # storage_qq = Z'Z (qxq)
    BLAS.trmm!('L', 'L', 'T', 'N', T(1), L, obs.storage_qq) # O(q^3); L'Z'Z
    BLAS.trmm!('R', 'L', 'N', 'N', T(1), L, obs.storage_qq) # O(q^3); L'Z'ZL
    @inbounds for j in 1:q
        obs.storage_qq[j, j] += σ²
    end
    # cholesky on M = σ² * I + Lt Zt Z L
    LAPACK.potrf!('U', obs.storage_qq) # O(q^3), store upper triangular matrix 
    # storage_q = (Mchol.U') \ (Lt * (Zt * res))
    # Z'(y - xβ) = Z'r
    BLAS.gemv!('N', T(-1), obs.ztx, β, T(1), copy!(obs.storage_q, obs.zty))
    BLAS.trmv!('L', 'T', 'N', L, obs.storage_q) # O(q^2); L'Z'r
    BLAS.trsv!('U', 'T', 'N', obs.storage_qq, obs.storage_q) # (A')⁻¹L'Z'r
    # l2 norm of residual vector
    # storage_p = X'Xβ - 2X'y
    copy!(obs.storage_p, obs.xty)
    rtr  = obs.yty +
        dot(β, BLAS.gemv!('N', T(1), obs.xtx, β, T(-2), obs.storage_p))
    # assemble pieces
    logl::T = n * log(2π) + (n - q) * log(σ²) # constant term
    @inbounds for j in 1:q
        logl += 2log(obs.storage_qq[j, j])
    end
    qf = abs2(norm(obs.storage_q)) # quadratic form term
    logl += (rtr - qf) / σ² 
    logl /= -2
    ###################
    # Evaluate gradient
    ###################    
    if needgrad
        
        # ∇β, gradient of β = X'Ω⁻¹r
        
        mul!(obs.xtxbeta, obs.xtx, β) # X'Xβ (px1)
        copy!(obs.storage_q2, obs.storage_q) # storage_q = (A')⁻¹L'Z'r (qx1)
        # A⁻¹(A')⁻¹L'Z'r (qx1)
        BLAS.trsv!('U', 'N', 'N', obs.storage_qq, obs.storage_q2) 
        # LA⁻¹(A')⁻¹L'Z'r (qx1)
        BLAS.trmv!('L', 'N', 'N', L, obs.storage_q2)
        # X'ZLA⁻¹(A')⁻¹L'Z'r (px1)
        mul!(obs.∇β, transpose(obs.ztx), obs.storage_q2) 
        obs.∇β .= (obs.xty .- obs.xtxbeta .- obs.∇β) ./ σ²
        
        # ∇σ², gradient of σ² = - (tr(Ω⁻¹) + r'Ω⁻²r) / 2
        
        copy!(obs.store_qq2, L)
        # LA⁻¹ 
        BLAS.trsm!('R', 'U', 'N', 'N', T(1), obs.storage_qq, obs.store_qq2) 
        # LA⁻¹(A')⁻¹L'
        mul!(obs.store_qq2, obs.store_qq2, transpose(obs.store_qq2)) 
        # LA⁻¹(A')⁻¹L'Z'Z
        mul!(obs.store_qq3, obs.store_qq2, obs.ztz) 
        trace = (n - tr(obs.store_qq3)) / σ²
        # Z'ZLA⁻¹(A')⁻¹L'Z'r
        mul!(obs.storage_q3, obs.ztz, obs.storage_q2) 
        obs.∇σ² .= - trace / 2 + 
            (rtr - 2qf + dot(obs.storage_q2, obs.storage_q3)) / 2(σ²)^2 
        
        # ∇L, gradient of L = -Z'Ω⁻¹ZL + Z'Ω⁻¹rr'Ω⁻¹ZL
        
        mul!(obs.ztxbeta, obs.ztx, β) # Z'Xβ
        # Z'Ω⁻¹r (qx1)
        obs.storage_q3 .= (obs.zty .- obs.ztxbeta .- obs.storage_q3) ./ σ² 
        # Z'Ω⁻¹rr'Ω⁻¹Z (qxq)
        mul!(obs.∇Σ, obs.storage_q3, transpose(obs.storage_q3))
        # Z'Ω⁻¹rr'Ω⁻¹ZL (qxq)
        BLAS.trmm!('R', 'L', 'N', 'N', T(1), L, obs.∇Σ) 
        @inbounds for i = 1:q
            obs.store_qq3[i, i] -= 1 # -I + LA⁻¹(A')⁻¹L'Z'Z
        end
        # -L + LA⁻¹(A')⁻¹L'Z'ZL
        BLAS.trmm!('R', 'L', 'N', 'N', T(1), L, obs.store_qq3)
        # -Z'ZL + Z'ZLA⁻¹(A')⁻¹L'Z'ZL
        BLAS.gemm!('N', 'N', T(1/σ²), obs.ztz, obs.store_qq3, T(1), obs.∇Σ) 
    end    
    ###################
    # Return
    ###################        
    return logl  
end

logl!

It is a good idea to test correctness and efficiency of the single datum objective/gradient evaluator here. First generate the same data set as in HW2.

In [5]:
Random.seed!(257)
# dimension
n, p, q = 2000, 5, 3
# predictors
X  = [ones(n) randn(n, p - 1)]
Z  = [ones(n) randn(n, q - 1)]
# parameter values
β  = [2.0; -1.0; rand(p - 2)]
σ² = 1.5
Σ  = fill(0.1, q, q) + 0.9I # compound symmetry 
L  = Matrix(cholesky(Symmetric(Σ)).L)
# generate y
y  = X * β + Z * rand(MvNormal(Σ)) + sqrt(σ²) * randn(n)

# form the LmmObs object
obs = LmmObs(y, X, Z);

### Correctness

In [6]:
@show logl = logl!(obs, β, L, σ², true)
@show obs.∇β
@show obs.∇σ²
@show obs.∇Σ;

logl = logl!(obs, β, L, σ², true) = -3256.179335805826
obs.∇β = [0.2669810805713458, 41.61418337067329, -34.34664962312693, 36.10898510707528, 27.913948208793148]
obs.∇σ² = [1.6283715138411026]
obs.∇Σ = [-0.9709131782279856 0.030145913774882682 -0.29967919774741214; -0.013843554536180348 -0.9701196695226567 0.281913321328081; -0.15698601809444018 0.38891970710313756 1.1593553981650055]


You will lose all 20 points if following statement throws `AssertionError`.

In [7]:
@assert abs(logl - (-3256.1793358058258)) < 1e-4
@assert norm(obs.∇β - [0.26698108057144054, 41.61418337067327, 
        -34.34664962312689, 36.10898510707527, 27.913948208793144]) < 1e-4
# @assert norm(obs.∇Σ - 
#    [-0.9464482950697888 0.057792444809492895 -0.30244127639188767; 
#        0.057792444809492895 -1.00087164917123 0.2845116557144694; 
#        -0.30244127639188767 0.2845116557144694 1.170040927259726]) < 1e-4
@assert abs(obs.∇σ²[1] - (1.6283715138412163)) < 1e-4

### Efficiency

Benchmark for evaluating objective only. This is what we did in HW3.

In [8]:
@benchmark logl!($obs, $β, $L, $σ², false)

BenchmarkTools.Trial: 10000 samples with 10 evaluations.
 Range (min … max):  1.307 μs …  4.600 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.321 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.337 μs ± 83.961 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

   ▃█                                                         
  ▂██▅▂▂▂▃█▄▂▂▁▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▁▁▂▁▁▁▂▂▂▂▂▁▂▁▁▂▂ ▂
  1.31 μs        Histogram: frequency by time        1.63 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

Benchmark for objective + gradient evaluation.

In [9]:
bm_objgrad = @benchmark logl!($obs, $β, $L, $σ², true)

BenchmarkTools.Trial: 10000 samples with 8 evaluations.
 Range (min … max):  3.552 μs …  14.047 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     3.606 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   3.622 μs ± 249.157 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

   ▁▄    █                                                     
  ▃██▆▆▆██▆▄▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▁▂▂▂▁▂▂▂▂▂▂▂▂▂▁▂▁▂▂▁▂▂▁▁▂▁▁▁▂ ▃
  3.55 μs         Histogram: frequency by time        4.09 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

My median runt time is 2.1μs. You will get full credit (10 pts) if the median run time is within 10μs.

In [10]:
#  The points you will get are
clamp(10 / (median(bm_objgrad).time / 1e3) * 10, 0, 10)

10.0

## Q3. LmmModel type

We create a `LmmModel` type to hold all data points and model parameters. Log-likelihood/gradient of a `LmmModel` object is simply the sum of log-likelihood/gradient of individual data points. 

In [11]:
# define a type that holds LMM model (data + parameters)
struct LmmModel{T <: AbstractFloat} <: MOI.AbstractNLPEvaluator
    # data
    data :: Vector{LmmObs{T}}
    # parameters
    β    :: Vector{T}
    L    :: Matrix{T}
    σ²   :: Vector{T}    
    # arrays for holding gradient
    ∇β   :: Vector{T}
    ∇σ²  :: Vector{T}
    ∇L   :: Matrix{T}
    # intermediate arrays for pre-allocation
    xty  :: Vector{T}
    ztr2 :: Vector{T}
    xtx  :: Matrix{T}
    ztz2 :: Matrix{T}
    ztzr :: Vector{T}
end

"""
    LmmModel(data::Vector{LmmObs})

Create an LMM model that contains data and parameters.
"""
function LmmModel(obsvec::Vector{LmmObs{T}}) where T <: AbstractFloat
    # dims
    p    = size(obsvec[1].X, 2)
    q    = size(obsvec[1].Z, 2)
    # parameters
    β    = Vector{T}(undef, p)
    L    = Matrix{T}(undef, q, q)
    σ²   = Vector{T}(undef, 1)    
    # gradients
    ∇β   = similar(β)    
    ∇σ²  = similar(σ²)
    ∇L   = similar(L)
    # intermediate arrays
    xty  = Vector{T}(undef, p)
    ztr2 = Vector{T}(undef, abs2(q))
    xtx  = Matrix{T}(undef, p, p)
    ztz2 = Matrix{T}(undef, abs2(q), abs2(q))
    ztzr = Vector{T}(undef, q)
    LmmModel(obsvec, β, L, σ², ∇β, ∇σ², ∇L, xty, ztr2, xtx, ztz2, ztzr)
end

"""
    logl!(m::LmmModel, needgrad=false)

Evaluate the log-likelihood of an LMM model at parameter values `m.β`, `m.L`, 
and `m.σ²`. If `needgrad==true`, then `m.∇β`, `m.∇Σ`, and `m.σ² are filled 
with the corresponding gradient.
"""
function logl!(m::LmmModel{T}, needgrad::Bool = false) where T <: AbstractFloat
    logl = zero(T)
    if needgrad
        fill!(m.∇β, 0)
        fill!(m.∇L, 0)
        fill!(m.∇σ², 0)        
    end
    @inbounds for i in 1:length(m.data)
        obs = m.data[i]
        logl += logl!(obs, m.β, m.L, m.σ²[1], needgrad)
        if needgrad
            BLAS.axpy!(T(1), obs.∇β, m.∇β)
            BLAS.axpy!(T(1), obs.∇Σ, m.∇L)
            m.∇σ²[1] += obs.∇σ²[1]
        end
    end
    logl
end

logl!

## Q4. (20 pts) Test data

Let's generate a fake longitudinal data set to test our algorithm.

In [22]:
Random.seed!(257)

# dimension
m      = 1000 # number of individuals
ns     = rand(1500:2000, m) # numbers of observations per individual
p      = 5 # number of fixed effects, including intercept
q      = 3 # number of random effects, including intercept
obsvec = Vector{LmmObs{Float64}}(undef, m)
# true parameter values
βtrue  = [0.1; 6.5; -3.5; 1.0; 5; zeros(p - 5)]
σ²true = 1.5
σtrue  = sqrt(σ²true)
Σtrue  = Matrix(Diagonal([2.0; 1.2; 1.0; zeros(q - 3)]))
Ltrue  = Matrix(cholesky(Symmetric(Σtrue), Val(true), check=false).L)
# generate data
for i in 1:m
    # first column intercept, remaining entries iid std normal
    X = Matrix{Float64}(undef, ns[i], p)
    X[:, 1] .= 1
    @views Distributions.rand!(Normal(), X[:, 2:p])
    # first column intercept, remaining entries iid std normal
    Z = Matrix{Float64}(undef, ns[i], q)
    Z[:, 1] .= 1
    @views Distributions.rand!(Normal(), Z[:, 2:q])
    # generate y
    y = X * βtrue .+ Z * (Ltrue * randn(q)) .+ σtrue * randn(ns[i])
    # form a LmmObs instance
    obsvec[i] = LmmObs(y, X, Z)
end
# form a LmmModel instance
lmm = LmmModel(obsvec);

For later comparison with other software, we save the data into a text file `lmm_data.csv`. **Do not put this file in Git.** It takes 245.4MB storage.

In [13]:
(isfile("lmm_data.csv") && filesize("lmm_data.csv") == 245369936) || 
open("lmm_data.csv", "w") do io
    p = size(lmm.data[1].X, 2)
    q = size(lmm.data[1].Z, 2)
    # print header
    print(io, "ID,Y,")
    for j in 1:(p - 1)
        print(io, "X" * string(j) * ",")
    end
    for j in 1:(q - 1)
        print(io, "Z" * string(j) * (j < q - 1 ? "," : "\n"))
    end
    # print data
    for i in eachindex(lmm.data)
        obs = lmm.data[i]
        for j in 1:length(obs.y)
            # id
            print(io, i, ",")
            # Y
            print(io, obs.y[j], ",")
            # X data
            for k in 2:p
                print(io, obs.X[j, k], ",")
            end
            # Z data
            for k in 2:(q - 1)
                print(io, obs.Z[j, k], ",")
            end
            print(io, obs.Z[j, q], "\n")
        end
    end
end

### Correctness

Evaluate log-likelihood and gradient of whole data set at the true parameter values.

In [23]:
copy!(lmm.β, βtrue)
copy!(lmm.L, Ltrue)
lmm.σ²[1] = σ²true
@show obj = logl!(lmm, true)
@show lmm.∇β
@show lmm.∇σ²
@show lmm.∇L;

obj = logl!(lmm, true) = -2.84006843836997e6
lmm.∇β = [41.06591670742584, 445.7512035397389, 157.0133992249181, -335.0997736073392, -895.6257448385876]
lmm.∇σ² = [-489.53617303824456]
lmm.∇L = [-3.3982575935323593 31.321038420873773 26.736450897360026; 40.43528672998911 61.86377650459801 -75.37427770753195; 37.81105146876874 -82.56838431214825 -56.459925427580394]


Test correctness. You will loss all 20 points if following code throws `AssertError`.

In [15]:
@assert abs(obj - (-2.840068438369969e6)) < 1e-4
@assert norm(lmm.∇β - [41.0659167074073, 445.75120353972426, 
        157.0133992249258, -335.09977360733626, -895.6257448385899]) < 1e-4
@assert norm(lmm.∇L - [-3.3982575935824837 31.32103842086001 26.73645089732865; 
        40.43528672997116 61.86377650461202 -75.37427770754684; 
        37.811051468724486 -82.56838431216435 -56.45992542754974]) < 1e-4
@assert abs(lmm.∇σ²[1] - (-489.5361730382465)) < 1e-4

### Efficiency

Test efficiency.

In [16]:
bm_model = @benchmark logl!($lmm, true)

BenchmarkTools.Trial: 1122 samples with 1 evaluation.
 Range (min … max):  4.078 ms …  10.854 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     4.240 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   4.414 ms ± 460.714 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

   ▆██▇▆▅▄▄▃▂▃▂▁▁▁▁                                           ▁
  ███████████████████████▇▇▆▆█▇▇█▆▇██▇▅▇▆▆▇▄▅▆▇▄▁▅▆▆▅▅▄▅▅▅▆▄▄ █
  4.08 ms      Histogram: log(frequency) by time      5.97 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

My median run time is 3.5ms. You will get full credit if your median run time is within 10ms. The points you will get are

In [17]:
clamp(10 / (median(bm_model).time / 1e6) * 10, 0, 10)

10.0

### Memory

You will lose 1 point for each 100 bytes memory allocation. So the points you will get is

In [18]:
clamp(10 - median(bm_model).memory / 100, 0, 10)

10.0

## Q5. (30 pts) Starting point

For numerical optimization, a good starting point is critical. Let's start $\boldsymbol{\beta}$ and $\sigma^2$ from the least sqaures solutions (ignoring intra-individual correlations)
\begin{eqnarray*}
\boldsymbol{\beta}^{(0)} &=& \left(\sum_i \mathbf{X}_i^T \mathbf{X}_i\right)^{-1} \left(\sum_i \mathbf{X}_i^T \mathbf{y}_i\right) \\
\sigma^{2(0)} &=& \frac{\sum_i \|\mathbf{r}_i^{(0)}\|_2^2}{\sum_i n_i} = \frac{\sum_i \|\mathbf{y}_i - \mathbf{X}_i \boldsymbol{\beta}^{(0)}\|_2^2}{\sum_i n_i}.
\end{eqnarray*}
To get a reasonable starting point for $\boldsymbol{\Sigma} = \mathbf{L} \mathbf{L}^T$, we can minimize the least squares criterion (ignoring the noise variance component)
$$
    \text{minimize} \sum_i \| \mathbf{r}_i^{(0)} \mathbf{r}_i^{(0)T} - \mathbf{Z}_i \boldsymbol{\Sigma} \mathbf{Z}_i^T \|_{\text{F}}^2.
$$
Derive the minimizer $\boldsymbol{\Sigma}^{(0)}$ (10 pts). 

We implement this start point strategy in the function `init_ls()`.

#### Solution to the minimizer $\mathbf{\Sigma}^{(0)}$

$$\sum_i \| \mathbf{r}_i^{(0)} \mathbf{r}_i^{(0)T} - \mathbf{Z}_i \boldsymbol{\Sigma} \mathbf{Z}_i^T \|_{\text{F}}^2 \\ = \sum_i \text{tr}((\mathbf{r}_i^{(0)} \mathbf{r}_i^{(0)T} - \mathbf{Z}_i \boldsymbol{\Sigma} \mathbf{Z}_i^T) (\mathbf{r}_i^{(0)} \mathbf{r}_i^{(0)T} - \mathbf{Z}_i \boldsymbol{\Sigma} \mathbf{Z}_i^T)^T) \\ = \sum_i \text{tr}((\mathbf{r}_i^{(0)} \mathbf{r}_i^{(0)T} - \mathbf{Z}_i \boldsymbol{\Sigma} \mathbf{Z}_i^T)^T (\mathbf{r}_i^{(0)} \mathbf{r}_i^{(0)T} - \mathbf{Z}_i \boldsymbol{\Sigma} \mathbf{Z}_i^T)) \\ = \sum_i \text{tr}(\mathbf{V}^T \mathbf{V})$$

where $\mathbf{V} = \mathbf{r}_i^{(0)} \mathbf{r}_i^{(0)T} - \mathbf{Z}_i \boldsymbol{\Sigma} \mathbf{Z}_i^T$.

To find $\mathbf{\Sigma}^{(0)}$, we minimize this function by taking the gradient with respect to $\mathbf{\Sigma}$ and setting it equal to zero. We then solve for $\mathbf{\Sigma}$.

$$D_{\mathbf{\Sigma}} \sum_i \text{tr}(\mathbf{V}^T \mathbf{V}) = D_{\mathbf{V}} \sum_i \text{tr}(\mathbf{V}^T \mathbf{V}) * D_{\mathbf{\Sigma}} \mathbf{V} \\ = \sum_i \text{vec}(2\mathbf{V})^T * -\mathbf{Z}_i \otimes \mathbf{Z}_i \\ = -2 \sum_i [(\mathbf{Z}_i^T \otimes \mathbf{Z}_i^T) \text{vec}(\mathbf{V})]^T \\ = -2 \sum_i \text{vec}(\mathbf{Z}_i^T \mathbf{V} \mathbf{Z}_i)^T$$

Plugging the value of $\mathbf{V}$ back in, we get $$ -2 \sum_i \text{vec}(\mathbf{Z}_i^T (\mathbf{r}_i^{(0)} \mathbf{r}_i^{(0)T} - \mathbf{Z}_i \boldsymbol{\Sigma} \mathbf{Z}_i^T) \mathbf{Z}_i)^T \\ = -2 \sum_i \text{vec}(\mathbf{Z}_i^T \mathbf{r}_i^{(0)} \mathbf{r}_i^{(0)T} \mathbf{Z}_i - \mathbf{Z}_i^T \mathbf{Z}_i \boldsymbol{\Sigma} \mathbf{Z}_i^T \mathbf{Z}_i)^T$$

Setting this function to 0, we get $$ \sum_i \text{vec}(\mathbf{Z}_i^T \mathbf{r}_i^{(0)} \mathbf{r}_i^{(0)T} \mathbf{Z}_i) = \sum_i \text{vec}(\mathbf{Z}_i^T \mathbf{Z}_i \boldsymbol{\Sigma}^{(0)} \mathbf{Z}_i^T \mathbf{Z}_i) \\ \sum_i \text{vec}((\mathbf{Z}_i^T \mathbf{r}_i^{(0)}) (\mathbf{Z}_i^T \mathbf{r}_i^{(0)})^T) = \sum_i (\mathbf{Z}_i^T \mathbf{Z}_i \otimes \mathbf{Z}_i^T \mathbf{Z}_i) \text{vec}(\boldsymbol{\Sigma}^{(0)}) \\ \sum_i (\mathbf{Z}_i^T \mathbf{r}_i^{(0)} \otimes \mathbf{Z}_i^T \mathbf{r}_i^{(0)}) = \sum_i (\mathbf{Z}_i^T \mathbf{Z}_i \otimes \mathbf{Z}_i^T \mathbf{Z}_i) \text{vec}(\boldsymbol{\Sigma}^{(0)})$$

Therefore, $$ \text{vec}(\boldsymbol{\Sigma}^{(0)}) = [\sum_i (\mathbf{Z}_i^T \mathbf{Z}_i \otimes \mathbf{Z}_i^T \mathbf{Z}_i)]^{-1} \sum_i (\mathbf{Z}_i^T \mathbf{r}_i^{(0)} \otimes \mathbf{Z}_i^T \mathbf{r}_i^{(0)}) $$

#### Solution to implementation

`kron_axpy!` given to me by Dr. Zhou to reduce allocations, improving runtime

In [19]:
"""
    kron_axpy!(A, X, Y)

Overwrite `Y` with `A ⊗ X + Y`. Same as `Y += kron(A, X)` but
more memory efficient.
"""
function kron_axpy!(
        A::AbstractVecOrMat{T},
        X::AbstractVecOrMat{T},
        Y::AbstractVecOrMat{T}
        ) where T <: Real
    m, n = size(A, 1), size(A, 2)
    p, q = size(X, 1), size(X, 2)
    @assert size(Y, 1) == m * p
    @assert size(Y, 2) == n * q
    @inbounds for j in 1:n
        coffset = (j - 1) * q
        for i in 1:m
            a = A[i, j]
            roffset = (i - 1) * p            
            for l in 1:q
                r = roffset + 1
                c = coffset + l
                for k in 1:p                
                    Y[r, c] += a * X[k, l]
                    r += 1
                end
            end
        end
    end
    Y
end

kron_axpy!

In [20]:
"""
    init_ls!(m::LmmModel)

Initialize parameters of a `LmmModel` object from the least squares estimate. 
`m.β`, `m.L`, and `m.σ²` are overwritten with the least squares estimates.
"""
function init_ls!(m::LmmModel{T}) where T <: AbstractFloat
    p, q = size(m.data[1].X, 2), size(m.data[1].Z, 2)
    
    # allocations
    n = 0
    sumrtr = 0
    
    # starting point for β⁰
    @inbounds for i in 1:length(m.data)
        obs = m.data[i]
        m.xtx .+= obs.xtx
        m.xty .+= obs.xty
        n += size(obs.X, 1)
    end
    m.β .= m.xtx \ m.xty
    
    # starting point for σ²⁽⁰⁾
    @inbounds for i in 1:length(m.data)
        obs = m.data[i]
        mul!(obs.storage_p, obs.xtx, m.β) # X'Xβ⁽⁰⁾
        sumrtr += obs.yty - 2dot(m.β, obs.xty) + dot(m.β, obs.storage_p)
        mul!(obs.ztxbeta, obs.ztx, m.β) # Z'Xβ⁽⁰⁾
        m.ztzr .= obs.zty .- obs.ztxbeta # Z'r⁽⁰⁾
        # m.ztr2 .+= kron(m.ztzr, m.ztzr) 
        kron_axpy!(m.ztzr, m.ztzr, m.ztr2) # Z'r ⊗ Z'r
        # m.ztz2 .+= kron(obs.ztz, obs.ztz) 
        kron_axpy!(obs.ztz, obs.ztz, m.ztz2) # Z'Z ⊗ Z'Z
    end
    m.σ²[1] = sumrtr / n 
    
    # starting point for L⁰
    LAPACK.potrf!('U', m.ztz2) # upper triangle of Cholesky Z'Z ⊗ Z'Z (q²xq²)
    BLAS.trsv!('U', 'T', 'N', m.ztz2, m.ztr2) # (A')⁻¹b
    BLAS.trsv!('U', 'N', 'N', m.ztz2, m.ztr2) # (A'A)⁻¹b = vec(Σ)
    m.L .= reshape(m.ztr2, q, q) # convert vec(Σ) into matrix (qxq)
    LAPACK.potrf!('L', m.L)
end

init_ls!

In [25]:
init_ls!(lmm)
@show logl!(lmm)
@show lmm.β
@show lmm.σ²
@show lmm.L;

logl!(lmm) = -3.362622488557282e6
lmm.β = [0.1820916846825799, 6.500470988235487, -3.4979075180177563, 1.0011139446410662, 4.991506621869463]
lmm.σ² = [5.709081253959118]
lmm.L = [1.40691980615069 0.07260414044838093 0.05717825295199748; 0.05160503116262733 1.13196516344654 -0.0770822321947434; 0.04064073354272282 -0.06994870608124523 0.9718144862989262]


### Correctness

Your start points should have a log-likelihood larger than -3.352991e6 (10 pts). The points you get are

In [26]:
# this is the points you get
(logl!(lmm) >  -3.3627e6) * 10

10

### Efficiency

The start point should be computed quickly. Otherwise there is no point using it as a starting point. You get full credit (10 pts) if the median run time is within 1ms.

In [27]:
bm_init = @benchmark init_ls!($lmm)

BenchmarkTools.Trial: 7377 samples with 1 evaluation.
 Range (min … max):  615.075 μs …  2.793 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     640.635 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   664.554 μs ± 85.291 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▃▇█▇▇▆▅▄▄▄▃▄▃▃▂▂▂▁▁▁▁▁                                       ▂
  ████████████████████████▇██▇▇█▇█▇██▇▇▇▆▇▅▇▇▆▆▆▆▆▅▆▅▅▅▅▅▅▁▃▁▄ █
  615 μs        Histogram: log(frequency) by time         1 ms <

 Memory estimate: 544 bytes, allocs estimate: 5.

In [28]:
# this is the points you get
clamp(1 / (median(bm_init).time / 1e6) * 10, 0, 10)

10.0

## Q6. NLP via MathOptInterface.jl

We define the NLP problem using the modelling tool [MathOptInterface.jl](https://github.com/jump-dev/MathOptInterface.jl). Start-up code is given below. Modify if necessary to accomodate your own code.

In [48]:
"""
    fit!(m::LmmModel, solver=Ipopt.Optimizer())

Fit an `LmmModel` object by MLE using a nonlinear programming solver. Start
point should be provided in `m.β`, `m.σ²`, `m.L`.
"""
function fit!(
        m :: LmmModel{T},
        solver = Ipopt.Optimizer()
    ) where T <: AbstractFloat
    p    = size(m.data[1].X, 2)
    q    = size(m.data[1].Z, 2)
    npar = p + ((q * (q + 1)) >> 1) + 1
    # prep the MOI
    MOI.empty!(solver)
    # set lower bounds and upper bounds of parameters
    # q diagonal entries of Cholesky factor L should be >= 0
    # σ² should be >= 0
    lb   = fill(0.0, q + 1)
    ub   = fill(Inf, q + 1)
    NLPBlock = MOI.NLPBlockData(MOI.NLPBoundsPair.(lb, ub), m, true)
    MOI.set(solver, MOI.NLPBlock(), NLPBlock)
    # start point
    params = MOI.add_variables(solver, npar)    
    par0   = Vector{T}(undef, npar)
    modelpar_to_optimpar!(par0, m)    
    for i in 1:npar
        MOI.set(solver, MOI.VariablePrimalStart(), params[i], par0[i])
    end
    MOI.set(solver, MOI.ObjectiveSense(), MOI.MAX_SENSE)
    # optimize
    MOI.optimize!(solver)
    optstat = MOI.get(solver, MOI.TerminationStatus())
    optstat in (MOI.LOCALLY_SOLVED, MOI.ALMOST_LOCALLY_SOLVED) || 
        @warn("Optimization unsuccesful; got $optstat")
    # update parameters and refresh gradient
    xsol = [MOI.get(solver, MOI.VariablePrimal(), 
            MOI.VariableIndex(i)) for i in 1:npar]
    optimpar_to_modelpar!(m, xsol)
    logl!(m, true)
    m
end

"""
    modelpar_to_optimpar!(par, m)

Translate model parameters in `m` to optimization variables in `par`.
"""
function modelpar_to_optimpar!(
        par :: Vector,
        m   :: LmmModel
    )
    p = size(m.data[1].X, 2)
    q = size(m.data[1].Z, 2)
    # β
    copyto!(par, m.β)
    # L
    offset = p + 1
    @inbounds for j in 1:q, i in j:q
        par[offset] = m.L[i, j]
        offset += 1
    end
    # σ²
    par[end] = m.σ²[1]
    par
end

"""
    optimpar_to_modelpar!(m, par)

Translate optimization variables in `par` to the model parameters in `m`.
"""
function optimpar_to_modelpar!(
        m   :: LmmModel, 
        par :: Vector
    )
    p = size(m.data[1].X, 2)
    q = size(m.data[1].Z, 2)
    # β
    copyto!(m.β, 1, par, 1, p)
    # L
    fill!(m.L, 0)
    offset = p + 1
    @inbounds for j in 1:q, i in j:q
        m.L[i, j] = par[offset]
        offset += 1
    end
    # σ²
    m.σ²[1] = par[end]    
    m
end

function MOI.initialize(
        m                  :: LmmModel, 
        requested_features :: Vector{Symbol}
    )
    for feat in requested_features
        if !(feat in MOI.features_available(m))
            error("Unsupported feature $feat")
        end
    end
end

MOI.features_available(m::LmmModel) = [:Grad, :Hess, :Jac]

function MOI.eval_objective(
        m   :: LmmModel, 
        par :: Vector
    )
    optimpar_to_modelpar!(m, par)
    logl!(m, false) # don't need gradient here
end

function MOI.eval_objective_gradient(
        m    :: LmmModel, 
        grad :: Vector, 
        par  :: Vector
    )
    p = size(m.data[1].X, 2)
    q = size(m.data[1].Z, 2)
    optimpar_to_modelpar!(m, par) 
    obj = logl!(m, true)
    # gradient wrt β
    copyto!(grad, m.∇β)
    # gradient wrt L
    offset = p + 1
    @inbounds for j in 1:q, i in j:q
        grad[offset] = m.∇L[i, j]
        offset += 1
    end
    # gradient with respect to σ²
    grad[end] = m.∇σ²[1]
    # return objective
    obj
end

function MOI.eval_constraint(m::LmmModel, g, par)
    p = size(m.data[1].X, 2)
    q = size(m.data[1].Z, 2)
    gidx   = 1
    offset = p + 1
    @inbounds for j in 1:q, i in j:q
        if i == j
            g[gidx] = par[offset]
            gidx   += 1
        end
        offset += 1
    end
    g[end] = par[end]
    g
end

function MOI.jacobian_structure(m::LmmModel)
    p    = size(m.data[1].X, 2)
    q    = size(m.data[1].Z, 2)
    row  = collect(1:(q + 1))
    col  = Int[]
    offset = p + 1
    for j in 1:q, i in j:q
        (i == j) && push!(col, offset)
        offset += 1
    end
    push!(col, offset)
    [(row[i], col[i]) for i in 1:length(row)]
end

MOI.eval_constraint_jacobian(m::LmmModel, J, par) = fill!(J, 1)

function MOI.hessian_lagrangian_structure(m::LmmModel)
    p    = size(m.data[1].X, 2)
    q    = size(m.data[1].Z, 2)    
    q◺   = ◺(q)
    # we work on the upper triangular part of the Hessian
    arr1 = Vector{Int}(undef, ◺(p) + ◺(q◺) + q◺ + 1)
    arr2 = Vector{Int}(undef, ◺(p) + ◺(q◺) + q◺ + 1)
    # Hββ block
    idx  = 1    
    for j in 1:p, i in 1:j
        arr1[idx] = i
        arr2[idx] = j
        idx      += 1
    end
    # HLL block
    for j in 1:q◺, i in 1:j
        arr1[idx] = p + i
        arr2[idx] = p + j
        idx      += 1
    end
    # HLσ² block
    for i in (p + 1):(p + q◺)
        arr1[idx] = i
        arr2[idx] = p + q◺ + 1
        idx      += 1
    end
    # Hσ²σ² block
    arr1[idx] = p + q◺ + 1
    arr2[idx] = p + q◺ + 1
    [(arr1[i], arr2[i]) for i in 1:length(arr1)]
end

function MOI.eval_hessian_lagrangian(
        m   :: LmmModel, 
        H   :: AbstractVector{T},
        par :: AbstractVector{T}, 
        σ   :: T, 
        μ   :: AbstractVector{T}
    ) where {T}    
    p  = size(m.data[1].X, 2)
    q  = size(m.data[1].Z, 2)    
    q◺ = ◺(q)
    optimpar_to_modelpar!(m, par)
    logl!(m, true, true)
    # Hββ block
    idx = 1    
    @inbounds for j in 1:p, i in 1:j
        H[idx] = m.Hββ[i, j]
        idx   += 1
    end
    # HLL block
    @inbounds for j in 1:q◺, i in 1:j
        H[idx] = m.HLL[i, j]
        idx   += 1
    end
    # HLσ² block
    @inbounds for j in 1:q, i in j:q
        H[idx] = m.Hσ²L[i, j]
        idx   += 1
    end
    # Hσ²σ² block
    H[idx] = m.Hσ²σ²[1, 1]
    lmul!(σ, H)
end

## Q7. (20 pts) Test drive

Now we can run any NLP solver (supported by MathOptInterface.jl) to compute the MLE. For grading purpose, let's use the `:LD_MMA` ([Method of Moving Asymptotes](https://nlopt.readthedocs.io/en/latest/NLopt_Algorithms/#mma-method-of-moving-asymptotes-and-ccsa)) algorithm in NLopt.jl.

In [30]:
# initialize from least squares
init_ls!(lmm)
println("objective value at starting point: ", logl!(lmm)); println()

# NLopt (LD_MMA) obj. val = -2.8400587866501934e6
NLopt_solver = NLopt.Optimizer()
MOI.set(NLopt_solver, MOI.RawOptimizerAttribute("algorithm"), :LD_MMA)
@time fit!(lmm, NLopt_solver)

println("objective value at solution: $(logl!(lmm)))")
println("solution values:")
@show lmm.β
@show lmm.σ²
@show lmm.L * transpose(lmm.L)
println("gradient @ solution:")
@show lmm.∇β
@show lmm.∇σ²
@show lmm.∇L
@show norm([lmm.∇β; vec(LowerTriangular(lmm.∇L)); lmm.∇σ²])

objective value at starting point: -3.3626049335837825e6

  2.276026 seconds (2.77 M allocations: 146.254 MiB, 1.61% gc time, 88.38% compilation time)
objective value at solution: -2.840058786650215e6)
solution values:
lmm.β = [0.1814775966266405, 6.50038355406066, -3.499864289641798, 0.9997119258241745, 4.999229481866026]
lmm.σ² = [1.4987345758407702]
lmm.L * transpose(lmm.L) = [1.9836265987450448 0.06578014122060505 0.055171035371844986; 0.06578014122060505 1.2814412039960783 -0.0905965355714011; 0.055171035371844986 -0.0905965355714011 0.9434276643438054]
gradient @ solution:
lmm.∇β = [0.020716999444491457, -0.007651508148643238, -0.007674503668372523, -0.0006705790547343327, -0.0007020999198772415]
lmm.∇σ² = [-0.009272770854806822]
lmm.∇L = [-0.00016114212992313082 -0.003013436093352806 0.053751475646575536; 0.0019717398658838092 0.00020334979187119906 0.0013612698726620728; 0.07878083996169388 0.0006895916653869433 0.010680395276534416]
norm([lmm.∇β; vec(LowerTriangular(lmm.∇L)); 

0.08341754461982318

### Correctness

You get 10 points if the following code does not throw `AssertError`.

In [31]:
# objective at solution should be close enough to the optimal
@assert logl!(lmm) > -2.840059e6
# gradient at solution should be small enough
@assert norm([lmm.∇β; vec(LowerTriangular(lmm.∇L)); lmm.∇σ²]) < 0.1

### Efficiency

My median run time is 140ms. You get 10 points if your median time is within 1s(=1000ms).

In [32]:
NLopt_solver = NLopt.Optimizer()
MOI.set(NLopt_solver, MOI.RawOptimizerAttribute("algorithm"), :LD_MMA)
bm_mma = @benchmark fit!($lmm, $(NLopt_solver)) setup=(init_ls!(lmm))

BenchmarkTools.Trial: 22 samples with 1 evaluation.
 Range (min … max):  227.739 ms … 245.639 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     231.265 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   232.401 ms ±   4.009 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

     ▃   ▃█   ▃                                                  
  ▇▁▁█▁▁▇██▇▁▇█▇▇▁▁▁▁▇▁▇▁▁▇▁▁▇▁▁▁▁▇▁▇▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▇ ▁
  228 ms           Histogram: frequency by time          246 ms <

 Memory estimate: 18.23 KiB, allocs estimate: 527.

In [33]:
# this is the points you get
clamp(1 / (median(bm_mma).time / 1e9) * 10, 0, 10)

10.0

## Q8. (10 pts) Gradient free vs gradient-based methods vs hessian-based methods

Advantage of using a modelling tool such as MathOptInterface.jl is that we can easily switch the backend solvers. For a research problem, we never know beforehand which solver works best. 

Try different solvers in the NLopt.jl and Ipopt.jl packages. Compare the results in terms of run times (the shorter the better), objective values at solution (the larger the better), and gradients at solution (closer to 0 the better). Summarize what you find.

See this [page](https://nlopt.readthedocs.io/en/latest/NLopt_Algorithms/) for the descriptions of algorithms in NLopt.

Documentation for the Ipopt can be found [here](https://coin-or.github.io/Ipopt/).  

In [34]:
"""
    ◺(n::Integer)

Triangular number `n * (n + 1) / 2`.
"""
@inline ◺(n::Integer) = (n * (n + 1)) >> 1

◺

In [35]:
# vector of solvers to compare
solvers = ["NLopt (LN_COBYLA, gradient free)", 
    "NLopt (LD_MMA, gradient-based)", "Ipopt (L-BFGS)"]

function setup_solver(s::String)
    if s == "NLopt (LN_COBYLA, gradient free)"
        solver = NLopt.Optimizer()
        MOI.set(solver, MOI.RawOptimizerAttribute("algorithm"), :LN_COBYLA)
    elseif s == "NLopt (LD_MMA, gradient-based)"
        solver = NLopt.Optimizer()
        MOI.set(solver, MOI.RawOptimizerAttribute("algorithm"), :LD_MMA)
    elseif s == "Ipopt (L-BFGS)"
        solver = Ipopt.Optimizer()
        MOI.set(solver, MOI.RawOptimizerAttribute("print_level"), 0)
        MOI.set(solver, MOI.RawOptimizerAttribute("hessian_approximation"), 
            "limited-memory")
        MOI.set(solver, MOI.RawOptimizerAttribute("tol"), 1e-6)
    elseif s == "Ipopt (use FIM)"
        # Ipopt (use Hessian) obj val = -2.8400587866468e6
        solver = Ipopt.Optimizer()
        MOI.set(solver, MOI.RawOptimizerAttribute("print_level"), 0)        
    else
        error("unrecognized solver $s")
    end
    solver
end

# containers for results
runtime = zeros(length(solvers))
objvals = zeros(length(solvers))
gradnrm = zeros(length(solvers))

for i in 1:length(solvers)
    solver = setup_solver(solvers[i])
    bm = @benchmark fit!($lmm, $solver) setup = (init_ls!(lmm))
    runtime[i] = median(bm).time / 1e9
    objvals[i] = logl!(lmm, true)
    gradnrm[i] = norm([lmm.∇β; vec(LowerTriangular(lmm.∇L)); lmm.∇σ²])
end


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************



In [36]:
# display results
pretty_table(
    hcat(solvers, runtime, objvals, gradnrm),
    header = ["Solver", "Runtime", "Log-Like", "Gradiant Norm"],
    formatters = (ft_printf("%5.2f", 2), ft_printf("%8.8f", 3:4))
    )

┌──────────────────────────────────┬─────────┬───────────────────┬───────────────┐
│                           Solver │ Runtime │          Log-Like │ Gradiant Norm │
├──────────────────────────────────┼─────────┼───────────────────┼───────────────┤
│ NLopt (LN_COBYLA, gradient free) │    1.37 │ -2840080.51934556 │  371.83004390 │
│   NLopt (LD_MMA, gradient-based) │    0.23 │ -2840058.78665022 │    0.08341194 │
│                   Ipopt (L-BFGS) │    5.44 │ -2840058.78664680 │    0.00056307 │
└──────────────────────────────────┴─────────┴───────────────────┴───────────────┘


## Q9. (10 pts) Compare with existing art

Let's compare our method with lme4 package in R and MixedModels.jl package in Julia. Both lme4 and MixedModels.jl are developed mainly by Doug Bates. Summarize what you find.

In [38]:
method  = ["My method", "lme4", "MixedModels.jl"]
runtime = zeros(3)  # record the run times
loglike = zeros(3); # record the log-likelihood at MLE

### Your approach

In [39]:
solver = setup_solver("NLopt (LD_MMA, gradient-based)")
bm_257 = @benchmark fit!($lmm, $solver) setup=(init_ls!(lmm))
runtime[1] = (median(bm_257).time) / 1e9
loglike[1] = logl!(lmm)

-2.84005878665022e6

### lme4 

In [40]:
R"""
library(lme4)
library(readr)
library(magrittr)

testdata <- read_csv("lmm_data.csv")
"""

┌ Warning: RCall.jl: Loading required package: Matrix
└ @ RCall /Users/amisheth/.julia/packages/RCall/6kphM/src/io.jl:172
┌ Warning: RCall.jl: Parsed with column specification:
│ cols(
│   ID = col_double(),
│   Y = col_double(),
│   X1 = col_double(),
│   X2 = col_double(),
│   X3 = col_double(),
│   X4 = col_double(),
│   Z1 = col_double(),
│   Z2 = col_double()
│ )
└ @ RCall /Users/amisheth/.julia/packages/RCall/6kphM/src/io.jl:172


RObject{VecSxp}
# A tibble: 1,744,977 × 8
      ID        Y     X1      X2     X3      X4      Z1      Z2
   <dbl>    <dbl>  <dbl>   <dbl>  <dbl>   <dbl>   <dbl>   <dbl>
 1     1   9.52    0.202 -0.463   0.798  0.734   0.685  -0.570 
 2     1  24.4     1.59  -1.95    1.20   1.43    1.64    0.369 
 3     1  -1.99    0.378 -0.0367  1.63  -1.15   -0.818   2.83  
 4     1 -17.4    -1.88   0.375  -0.498 -0.253   1.56    1.68  
 5     1  -0.0704  0.658 -0.165   0.780 -1.23   -0.0288 -1.09  
 6     1  -0.853   0.458 -0.313  -0.512 -0.800  -0.331   1.98  
 7     1  -1.80    0.220  0.328   1.32  -1.01   -0.363  -0.0703
 8     1   5.88    1.30   0.889  -0.854  0.0714 -0.658  -0.0339
 9     1  -9.21   -1.43  -0.522  -0.119 -0.580  -0.155  -1.89  
10     1 -11.3    -0.468 -0.700   0.872 -1.82    1.80    0.492 
# … with 1,744,967 more rows


In [41]:
R"""
rtime <- system.time(mmod <- 
  lmer(Y ~ X1 + X2 + X3 + X4 + (1 + Z1 + Z2 | ID), testdata, REML = FALSE))
"""

┌ Warning: RCall.jl: Warning in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv,  :
│   Model failed to converge with max|grad| = 0.00434177 (tol = 0.002, component 1)
└ @ RCall /Users/amisheth/.julia/packages/RCall/6kphM/src/io.jl:172


RObject{RealSxp}
   user  system elapsed 
166.647  16.797 191.355 


In [42]:
R"""
rtime <- rtime["elapsed"]
summary(mmod)
rlogl <- logLik(mmod)
"""
runtime[2] = @rget rtime
loglike[2] = @rget rlogl;

### MixedModels.jl



In [43]:
testdata = CSV.File("lmm_data.csv", types = Dict(1=>String)) |> DataFrame

1744977×8 DataFrame
     Row │ ID      Y            X1          X2           X3         X4         ⋯
         │ String  Float64      Float64     Float64      Float64    Float64    ⋯
─────────┼──────────────────────────────────────────────────────────────────────
       1 │ 1         9.52102     0.201821   -0.463234     0.797731   0.73357   ⋯
       2 │ 1        24.4063      1.58557    -1.94608      1.19787    1.43149
       3 │ 1        -1.99215     0.378332   -0.0367002    1.63072   -1.15031
       4 │ 1       -17.4233     -1.8826      0.374561    -0.49786   -0.253248
       5 │ 1        -0.0704245   0.658283   -0.165487     0.77951   -1.22763   ⋯
       6 │ 1        -0.853357    0.457784   -0.313387    -0.512299  -0.800278
       7 │ 1        -1.80061     0.220461    0.327879     1.32209   -1.01336
       8 │ 1         5.88119     1.30135     0.88884     -0.853941   0.0714372
       9 │ 1        -9.20504    -1.43248    -0.521638    -0.119287  -0.579596  ⋯
      10 │ 1       -11.2909     -0.46827    -0.699709     0.871668  -1.81529
      11 │ 1         8.9367      2.00212     1.1577      -0.973746  -0.784991
    ⋮    │   ⋮          ⋮           ⋮            ⋮           ⋮          ⋮      ⋱
 1744968 │ 1000     -3.12427    -0.393863   -0.491481     1.55365   -0.776784
 1744969 │ 1000     -3.20226    -0.126944    0.601992    -0.960992  -0.300833  ⋯
 1744970 │ 1000      0.0979076  -0.0942822  -0.017406     1.01639   -0.111906
 1744971 │ 1000      6.84005     0.286095   -0.995131    -0.293129   0.193649
 1744972 │ 1000      8.44826    -0.0458988  -0.00257329  -1.31491    1.51216
 1744973 │ 1000     -5.13234    -1.11932    -1.06951     -0.339256  -0.500339  ⋯
 1744974 │ 1000     -0.530812   -0.850859    0.00500998  -0.274531   1.01461
 1744975 │ 1000     -8.88652    -0.762326    0.154867    -0.933923  -0.931906
 1744976 │ 1000     -1.42159    -0.672599   -1.25194     -0.518835  -0.0647723
 1744977 │ 1000      3.79889    -1.21758    -1.49033      1.04599    0.459857  ⋯
                                              2 columns and 1744956 rows omitted

In [44]:
mj = fit(MixedModel, 
    @formula(Y ~ X1 + X2 + X3 + X4 + (1 + Z1 + Z2 | ID)), testdata)
bm_mm = @benchmark fit(MixedModel, 
    @formula(Y ~ X1 + X2 + X3 + X4 + (1 + Z1 + Z2 | ID)), $testdata)
loglike[3] = loglikelihood(mj)
runtime[3] = median(bm_mm).time / 1e9

┌ Warning: ProgressMeter by default refresh meters with additional information in IJulia via `IJulia.clear_output`, which clears all outputs in the cell. 
│  - To prevent this behaviour, do `ProgressMeter.ijulia_behavior(:append)`. 
│  - To disable this warning message, do `ProgressMeter.ijulia_behavior(:clear)`.
└ @ ProgressMeter /Users/amisheth/.julia/packages/ProgressMeter/sN2xr/src/ProgressMeter.jl:618
Minimizing 212 	 Time: 0:00:00 ( 1.74 ms/it)


1.729697537

In [45]:
display(bm_mm)
mj

BenchmarkTools.Trial: 3 samples with 1 evaluation.
 Range (min … max):  1.380 s …    2.480 s  ┊ GC (min … max): 4.01% … 8.90%
 Time  (median):     1.730 s               ┊ GC (median):    8.37%
 Time  (mean ± σ):   1.863 s ± 562.163 ms  ┊ GC (mean ± σ):  7.53% ± 2.68%

  █                 █                                      █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  1.38 s         Histogram: frequency by time         2.48 s <

 Memory estimate: 910.54 MiB, allocs estimate: 1074252.

|             |    Est. |     SE |        z |      p |   σ_ID |
|:----------- | -------:| ------:| --------:| ------:| ------:|
| (Intercept) |  0.1815 | 0.0445 |     4.08 | <1e-04 | 1.4082 |
| X1          |  6.5004 | 0.0009 |  7000.80 | <1e-99 |        |
| X2          | -3.4999 | 0.0009 | -3768.92 | <1e-99 |        |
| X3          |  0.9997 | 0.0009 |  1078.16 | <1e-99 |        |
| X4          |  4.9992 | 0.0009 |  5391.08 | <1e-99 |        |
| Z2          |         |        |          |        | 0.9714 |
| Z1          |         |        |          |        | 1.1321 |
| Residual    |  1.2242 |        |          |        |        |


### Summary

In [47]:
pretty_table(
    hcat(method, runtime, loglike),
    header = ["Method", "Runtime", "Log-Like"],
    formatters = (ft_printf("%5.2f", 2), ft_printf("%8.6f", 3))
    )

┌────────────────┬─────────┬─────────────────┐
│         Method │ Runtime │        Log-Like │
├────────────────┼─────────┼─────────────────┤
│      My method │    0.27 │ -2840058.786650 │
│           lme4 │  191.36 │ -2840058.786659 │
│ MixedModels.jl │    1.73 │ -2840058.786722 │
└────────────────┴─────────┴─────────────────┘


## Q9. Be proud of yourself

Go to your resume/cv and claim you have experience performing analysis on complex longitudinal data sets with millions of records. And you beat current software by XXX fold.